# Klasifikasi Gambar

## Import Library

In [ ]:
import kagglehub
import numpy as np
import tensorflow as tf
import cv2
from PIL import Image

import zipfile
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Load Dataset

In [ ]:
kagglehub.login()

In [ ]:
# Download dataset
!kaggle datasets download -d ilhamhasib/vehicle-classification-dataset

Dataset URL: https://www.kaggle.com/datasets/ilhamhasib/vehicle-classification-dataset
License(s): CC-BY-SA-4.0




  0%|          | 0.00/4.38G [00:00<?, ?B/s]
  0%|          | 1.00M/4.38G [00:01<1:25:39, 915kB/s]
  0%|          | 2.00M/4.38G [00:01<1:01:25, 1.28MB/s]
  0%|          | 3.00M/4.38G [00:02<58:18, 1.34MB/s]  
  0%|          | 4.00M/4.38G [00:03<51:52, 1.51MB/s]
  0%|          | 5.00M/4.38G [00:03<51:49, 1.51MB/s]
  0%|          | 6.00M/4.38G [00:04<55:50, 1.40MB/s]
  0%|          | 7.00M/4.38G [00:04<45:50, 1.71MB/s]
  0%|          | 8.00M/4.38G [00:05<38:49, 2.01MB/s]
  0%|          | 9.00M/4.38G [00:05<34:42, 2.25MB/s]
  0%|          | 10.0M/4.38G [00:05<32:39, 2.39MB/s]
  0%|          | 11.0M/4.38G [00:06<30:52, 2.53MB/s]
  0%|          | 12.0M/4.38G [00:06<29:45, 2.63MB/s]
  0%|          | 13.0M/4.38G [00:07<29:55, 2.61MB/s]
  0%|          | 14.0M/4.38G [00:07<34:21, 2.27MB/s]
  0%|          | 15.0M/4.38G [00:08<36:25, 2.14MB/s]
  0%|          | 16.0M/4.38G [00:08<33:32, 2.33MB/s]
  0%|          | 17.0M/4.38G [00:08<30:43, 2.54MB/s]
  0%|          | 18.0M/4.38G [00:09<29:57, 2.61MB

In [ ]:
zip_path = 'vehicle-classification-dataset.zip'

extracted_folder_names = []  # More descriptive variable name

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    extracted_folders = zip_ref.namelist()
    for item in extracted_folders:
        if item.endswith('/'):  # Check for trailing slash, indicating a directory
            extracted_folder_names.append(item[:-1]) # Append the directory name without the trailing slash
            # Create the directory if it doesn't exist (important!)
            os.makedirs(item[:-1], exist_ok=True) # exist_ok prevents error if dir exists

    zip_ref.extractall()

In [ ]:
extracted_folder_names_after_extraction = [f for f in os.listdir() if os.path.isdir(f)]
print(extracted_folder_names_after_extraction)

['Car-Classification-Data-with-Labels-iHasib', 'Vehicle Test Data']


## Mengecek Resolusi Gambar

In [ ]:
# Path to the train directory
train_dir = 'Car-Classification-Data-with-Labels-iHasib/train'

# Dictionary to store image resolutions
image_resolutions = {}

# Iterate through each subfolder and image file
for subdir, _, files in os.walk(train_dir):
    for file in files:
        if file.endswith(('jpg', 'jpeg', 'png')):
            file_path = os.path.join(subdir, file)
            with Image.open(file_path) as img:
                image_resolutions[file_path] = img.size

# Print the resolutions
for path, resolution in image_resolutions.items():
    print(f'{path}: {resolution}')

KeyboardInterrupt: 

In [ ]:
# Load the labels
labels_df = pd.read_csv('Car-Classification-Data-with-Labels-iHasib/type-labels.csv')

# Create a dictionary mapping filenames to labels
labels_dict = dict(zip(labels_df['filename'], labels_df['label']))

# Define a function to get the label for a given filename
def get_label(filename):
    return labels_dict[filename]

# Create an ImageDataGenerator for training data
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Create a generator for the training data
train_generator = train_datagen.flow_from_directory(
    'Car-Classification-Data-with-Labels-iHasib/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

# Create a generator for the validation data
validation_generator = train_datagen.flow_from_directory(
    'Car-Classification-Data-with-Labels-iHasib/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

## Image Augmentation

In [ ]:
class ImageAugmentor:
    def __init__(self):
        pass
    
    def horizontal_augment(self, image):
        """Flip the image horizontally."""
        return cv2.flip(image, 1)
    
    def scaling_augment(self, image, scale_percent=70):
        """Scale the image by a given percentage."""
        width = int(image.shape[1] * scale_percent / 100)
        height = int(image.shape[0] * scale_percent / 100)
        dim = (width, height)
        return cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
    
    def translation_augment(self, image, tx=50, ty=50):
        """Translate the image by tx and ty pixels."""
        rows, cols, _ = image.shape
        M_translation = np.float32([[1, 0, tx], [0, 1, ty]])
        return cv2.warpAffine(image, M_translation, (cols, rows))
    
    def rotation_augment(self, image, angle=45):
        """Rotate the image by a given angle."""
        rows, cols, _ = image.shape
        M_rotation = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        return cv2.warpAffine(image, M_rotation, (cols, rows))
    
    def blur_augment(self, image, ksize=(5, 5)):
        """Apply Gaussian blur to the image."""
        return cv2.GaussianBlur(image, ksize, 0)
    
    def apply_all(self, image):
        """Apply all augmentations to the image and return a dictionary of results."""
        return {
            "horizontal": self.horizontal_augment(image),
            "scaled": self.scaling_augment(image),
            "translated": self.translation_augment(image),
            "rotated": self.rotation_augment(image),
            "blurred": self.blur_augment(image)
        }

In [ ]:

# Example usage
image_path = 'kucing.jpg'
image = cv2.imread(image_path)
imgAug = ImageAugmentor()
augmented_img = imgAug.apply_all(image)

# Display the augmented images
plt.figure(figsize=(10, 10))
plt.subplot(3, 2, 1)
plt.imshow(cv2.cvtColor(augmented_img["horizontal"], cv2.COLOR_BGR2RGB))
plt.title('Flipped')
plt.subplot(3, 2, 2)
plt.imshow(cv2.cvtColor(augmented_img["scaled"], cv2.COLOR_BGR2RGB))
plt.title('Scaled')
plt.subplot(3, 2, 3)
plt.imshow(cv2.cvtColor(augmented_img["translated"], cv2.COLOR_BGR2RGB))
plt.title('Translated')
plt.subplot(3, 2, 4)
plt.imshow(cv2.cvtColor(augmented_img["rotated"], cv2.COLOR_BGR2RGB))
plt.title('Rotated')
plt.subplot(3, 2, 5)
plt.imshow(cv2.cvtColor(augmented_img["blurred"], cv2.COLOR_BGR2RGB))
plt.title('Blurred')
plt.show()

## Define the Model

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (2,2), activation="relu"))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dense(3, activation="softmax"))

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Define Callback
from keras.callbacks import Callback

class StopTrainingCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get("accuracy") >= 0.95:
            print("Reached 95% accuracy. Stopping training...")
            self.model_stop_training = True